In [13]:
from dotenv import load_dotenv
import os
from main import (
    add_metric,
    remove_metric,
    get_metrics,
    validate_user_input,
    export_to_excel,
    create_human_review_template,
)

In [2]:
env_path = "/Users/ford/Documents/coding/confidential/.env"
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "API key is missing"

In [3]:
# Initialize default metrics
add_metric(
    "Have cause",
    "Text identifies and explains the underlying root cause or triggering factors of the risk",
)
add_metric(
    "Have risk event",
    "Text clearly describes what could go wrong or the specific risk scenario that may occur",
)
add_metric(
    "Have impact",
    "Text describes potential consequences or outcomes resulting from the risk event",
)

{'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [4]:
get_metrics()

{'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [15]:
user_inputs = [
    "ไม่จัดส่งรายงานพลังงานประจำปี",
    "ถุงจัมโบ้เสียหายในการจัดเก็บและขนส่ง ทำให้เกิดการปนของสายพันธุ์",
    "เครื่องจักร Brakedown บ่อย",
]

In [16]:
all_result = []
for user_input in user_inputs:
    results = validate_user_input(
        user_input,
        provider="openai",
        export=False,
    )
    all_result.extend(results)

Validating user input: 'ไม่จัดส่งรายงานพลังงานประจำปี'
Using provider: openai, model: default
Metrics: {'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk', 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur', 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}
Validating text on metric: Have cause
Validating text on metric: Have risk event
Validating text on metric: Have impact

Metric: Have cause - INVALID
Reason: The text 'ไม่จัดส่งรายงานพลังงานประจำปี' translates to 'Not submitting the annual energy report.' While it indicates a specific risk (failure to submit a report), it does not identify or explain the underlying root cause or triggering factors of this risk. To meet the validation criteria, the text should elaborate on why the report is not being submitted, such as lack of resources, oversight, or other contributing factor

In [17]:
human_template = create_human_review_template(all_result)

Human review template exported to human_review_template_openai_20250402_150422.xlsx
